In [1]:
import pandas as pd
import unicodedata
import matplotlib.pyplot as plt

file_path = "INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2025_A_31-07-2025.CSV"

# Ler CSV tirando as primeiras 8 linhas de cabeçalho
df = pd.read_csv(file_path, sep=";", encoding="utf-8", skiprows=8)

df.columns = [
    unicodedata.normalize("NFKD", col)
    .encode("ASCII", "ignore")
    .decode("utf-8")
    .strip()
    for col in df.columns
]

# Renomear colunas principais
renomear = {
    "data": ["Data"],
    "hora_utc": ["Hora UTC"],
    "chuva_mm": ["PRECIPITACAO TOTAL, HORARIO (mm)",
                 "PRECIPITAO TOTAL, HORRIO (mm)"],
    "temp_c": ["TEMPERATURA DO AR - BULBO SECO, HORARIA (C)"],
    "umidade": ["UMIDADE RELATIVA DO AR, HORARIA (%)"]
}

# Criar mapeamento dinâmico
col_map = {}
for novo, antigos in renomear.items():
    for old in antigos:
        if old in df.columns:
            col_map[old] = novo

df = df.rename(columns=col_map)

df["datetime"] = pd.to_datetime(
    df["data"] + " " + df["hora_utc"],
    format="%Y/%m/%d %H%M UTC",
    errors="coerce"
)

# Converter numéricos (corrigir vírgula decimal)
for col in ["chuva_mm", "temp_c", "umidade"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(",", ".").astype(float)

df["mes"] = df["datetime"].dt.month
df["ano"] = df["datetime"].dt.year

# Resumo diário para um mês escolhido
def resumo_diario(mes_escolhido):
    df_mes = df[df["mes"] == mes_escolhido]
    if df_mes.empty:
        return f"Nenhum dado disponível para o mês {mes_escolhido}."
    df_daily = df_mes.groupby(df_mes["datetime"].dt.date).agg({
        "chuva_mm": "sum" if "chuva_mm" in df_mes else "first",
        "temp_c": "mean" if "temp_c" in df_mes else "first",
        "umidade": "mean" if "umidade" in df_mes else "first"
    }).reset_index().rename(columns={"datetime": "dia"})
    return df_daily

mes_escolhido = int(input("Digite o número do mês para o resumo diário: "))

print("\nResumo diário:")
print(resumo_diario(mes_escolhido))

# Resumo mensal até o último mês disponível
df["mes_ano"] = df["datetime"].dt.to_period("M")
df_monthly = df.groupby("mes_ano").agg({
    "chuva_mm": "sum" if "chuva_mm" in df else "first",
    "temp_c": "mean" if "temp_c" in df else "first",
    "umidade": "mean" if "umidade" in df else "first"
}).reset_index()

print("\nResumo mensal:")
print(df_monthly)




Resumo diário:
           dia  chuva_mm     temp_c    umidade
0   2025-04-01       0.0  23.208333  73.833333
1   2025-04-02       0.0  25.220833  66.208333
2   2025-04-03      11.0  23.237500  76.458333
3   2025-04-04      26.6  22.345833  82.833333
4   2025-04-05       1.0  17.750000  80.208333
5   2025-04-06       0.2  17.775000  72.500000
6   2025-04-07       0.4  18.708333  70.833333
7   2025-04-08       0.0  19.812500  70.291667
8   2025-04-09       7.8  22.834783  68.347826
9   2025-04-10      24.6  20.779167  83.375000
10  2025-04-11       0.2  20.537500  75.625000
11  2025-04-12       0.0  20.686957  73.434783
12  2025-04-13       2.0  21.933333  69.958333
13  2025-04-14      58.6  20.212500  81.000000
14  2025-04-15       0.0  20.004167  73.375000
15  2025-04-16       0.8  21.120833  73.833333
16  2025-04-17       0.0  21.941667  72.500000
17  2025-04-18      11.2  21.504348  75.130435
18  2025-04-19       1.8  21.766667  78.666667
19  2025-04-20       5.6  20.050000  81.3333

In [ ]:
# Gráfico do resumo diário
df_daily = resumo_diario(mes_escolhido)
if not df_daily.empty:
    fig, ax1 = plt.subplots(figsize=(8, 4))
    
    # Barras de chuva
    ax1.bar(df_daily["dia"], df_daily["chuva_mm"], color="skyblue", alpha=0.6, label="Chuva (mm)")
    ax1.set_xlabel("Dia")
    ax1.set_ylabel("Chuva (mm)", color="skyblue")
    ax1.tick_params(axis='y', labelcolor="skyblue")

    # Linhas de temperatura e umidade
    ax2 = ax1.twinx()
    ax2.plot(df_daily["dia"], df_daily["temp_c"], color="orange", marker="o", label="Temperatura (°C)")
    ax2.plot(df_daily["dia"], df_daily["umidade"], color="blue", marker="o", label="Umidade (%)")
    ax2.set_ylabel("Temperatura/Umidade", color="black")
    fig.autofmt_xdate(rotation=45)
    fig.tight_layout()
    fig.legend(loc="upper right")
    plt.title(f"Resumo diário - Mês {mes_escolhido}")
    plt.show()

# Gráfico resumo mensal
fig, ax1 = plt.subplots(figsize=(8, 4))
ax1.bar(df_monthly["mes_ano"].astype(str), df_monthly["chuva_mm"], color="skyblue", alpha=0.6, label="Chuva (mm)")
ax1.set_xlabel("Mês/Ano")
ax1.set_ylabel("Chuva (mm)", color="skyblue")
ax1.tick_params(axis='y', labelcolor="skyblue")
ax2 = ax1.twinx()
ax2.plot(df_monthly["mes_ano"].astype(str), df_monthly["temp_c"], color="orange", marker="o", label="Temperatura (°C)")
ax2.plot(df_monthly["mes_ano"].astype(str), df_monthly["umidade"], color="blue", marker="o", label="Umidade (%)")
ax2.set_ylabel("Temperatura/Umidade", color="black")
fig.autofmt_xdate(rotation=45)
fig.tight_layout()
fig.legend(loc="upper right")
plt.title("Resumo mensal")
plt.show()

In [2]:
import plotly.express as px

# Gráfico do resumo diário
df_daily = resumo_diario(mes_escolhido)
if not df_daily.empty:
    fig = px.bar(
        df_daily,
        x="dia",
        y="chuva_mm",
        labels={"dia": "Dia", "chuva_mm": "Chuva (mm)"},
        title=f"Resumo diário - Mês {mes_escolhido}",
        color_discrete_sequence=["skyblue"]
    )
    fig.add_scatter(
        x=df_daily["dia"], y=df_daily["temp_c"],
        mode="lines+markers", name="Temperatura (°C)", yaxis="y2", line=dict(color="orange")
    )
    fig.add_scatter(
        x=df_daily["dia"], y=df_daily["umidade"],
        mode="lines+markers", name="Umidade (%)", yaxis="y2", line=dict(color="blue")
    )
    fig.update_layout(
        yaxis=dict(title="Chuva (mm)", color="skyblue"),
        yaxis2=dict(title="Temperatura/Umidade", overlaying="y", side="right"),
        legend=dict(x=0.01, y=0.99)
    )
    fig.show()

# Gráfico resumo mensal
fig = px.bar(
    df_monthly,
    x=df_monthly["mes_ano"].astype(str),
    y="chuva_mm",
    labels={"mes_ano": "Mês/Ano", "chuva_mm": "Chuva (mm)"},
    title="Resumo mensal",
    color_discrete_sequence=["skyblue"]
)
fig.add_scatter(
    x=df_monthly["mes_ano"].astype(str), y=df_monthly["temp_c"],
    mode="lines+markers", name="Temperatura (°C)", yaxis="y2", line=dict(color="orange")
)
fig.add_scatter(
    x=df_monthly["mes_ano"].astype(str), y=df_monthly["umidade"],
    mode="lines+markers", name="Umidade (%)", yaxis="y2", line=dict(color="blue")
)
fig.update_layout(
    yaxis=dict(title="Chuva (mm)", color="skyblue"),
    yaxis2=dict(title="Temperatura/Umidade", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99)
)
fig.show()